In [1]:
from google.colab import drive, files
import pandas as pd
drive.mount('/content/drive', force_remount=True)
train_path = "/content/dataset"

Mounted at /content/drive


In [2]:
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/ai_ml_suite/optiver-trading-at-the-close/train.csv.zip", 'r')
zip_ref.extractall(train_path)
zip_ref.close()

In [3]:
!pip install tensorflow
!pip install scikeras
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.callbacks import EarlyStopping
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification
from scikeras.wrappers import KerasRegressor
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.datasets import make_classification
from numba import njit, prange
import gc
from itertools import combinations
from sklearn.metrics import mean_absolute_error
import scipy

In [4]:
data = pd.read_csv('/content/dataset/train.csv')
data.drop(columns=['row_id'], inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5237980 entries, 0 to 5237979
Data columns (total 16 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   stock_id                 int64  
 1   date_id                  int64  
 2   seconds_in_bucket        int64  
 3   imbalance_size           float64
 4   imbalance_buy_sell_flag  int64  
 5   reference_price          float64
 6   matched_size             float64
 7   far_price                float64
 8   near_price               float64
 9   bid_price                float64
 10  bid_size                 float64
 11  ask_price                float64
 12  ask_size                 float64
 13  wap                      float64
 14  target                   float64
 15  time_id                  int64  
dtypes: float64(11), int64(5)
memory usage: 639.4 MB


In [ ]:
'''
physical_devices = tf.config.experimental.list_physical_devices("GPU")
print("Num GPUs Available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)
'''

'\nphysical_devices = tf.config.experimental.list_physical_devices("GPU")\nprint("Num GPUs Available: ", len(physical_devices))\ntf.config.experimental.set_memory_growth(physical_devices[0], True)\n'

In [5]:
def reduce_memory_usage(df):
    """
    Reduce memory usage by downcasting data types
    within a pandas DataFrame.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print(f'Memory usage of dataframe is {start_mem:.2f} MB')

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print(f'Memory usage after optimization is: {end_mem:.2f} MB')
    print(f'Decreased by {100 * (start_mem - end_mem) / start_mem:.1f}%')

    return df




# Zeroing the nulls for far price and near price
data.loc[(data['seconds_in_bucket']<300)&(data['far_price'].isna()), 'far_price'] = 0
data.loc[(data['seconds_in_bucket']<300)&(data['near_price'].isna()), 'near_price'] = 0


# Find rows with null values
# rows_with_null = data[data.isnull().any(axis=1)]


# Display rows with null values
# print(rows_with_null)

cols_with_null = data.columns[data.isnull().any()]

# Display columns with null values
print(cols_with_null)
cols_with_null = ['imbalance_size', 'reference_price', 'matched_size', 'far_price', 'near_price', 'bid_price', 'ask_price', 'wap', 'target']
for col in cols_with_null:
  null_counts = data[col].isnull().sum()
  # Display columns and their corresponding null counts
  print(col, null_counts)

for col in cols_with_null:
  rolling_mean = data[col].rolling(window=120, min_periods=1).mean()
  # Fill null values with rolling average of previous 10 non-null entries
  filled_values = data[col].fillna(rolling_mean)
  # Update the original DataFrame with filled values
  data[col] = filled_values

print('Post Cleaning\n')
for col in cols_with_null:
  null_counts = data[col].isnull().sum()
  # Display columns and their corresponding null counts
  print(col, null_counts)


#data.dropna(inplace = True)

Index(['imbalance_size', 'reference_price', 'matched_size', 'far_price',
       'near_price', 'bid_price', 'ask_price', 'wap', 'target'],
      dtype='object')
imbalance_size 220
reference_price 220
matched_size 220
far_price 37262
near_price 100
bid_price 220
ask_price 220
wap 220
target 88
Post Cleaning

imbalance_size 0
reference_price 0
matched_size 0
far_price 0
near_price 0
bid_price 0
ask_price 0
wap 0
target 0


In [6]:
categorical_columns = []
for col in data.columns:
      if data[col].dtype == 'object' or data[col].nunique() < len(data) / 4:
          categorical_columns.append(col)
print(data[categorical_columns].head(16))
# One-hot encode the 'imbalance_buy_sell_flag' column
one_hot_encoded = pd.get_dummies(data['imbalance_buy_sell_flag'], prefix='imbalance_buy_sell_flag')

# Concatenate the one-hot encoded columns with the original DataFrame
data = pd.concat([data, one_hot_encoded], axis=1)

# Drop the original 'imbalance_buy_sell_flag' column if needed
data.drop('imbalance_buy_sell_flag', axis=1, inplace=True)

    stock_id  date_id  seconds_in_bucket  imbalance_buy_sell_flag  \
0          0        0                  0                        1   
1          1        0                  0                       -1   
2          2        0                  0                       -1   
3          3        0                  0                       -1   
4          4        0                  0                       -1   
5          5        0                  0                        0   
6          6        0                  0                        1   
7          7        0                  0                        1   
8          8        0                  0                        1   
9          9        0                  0                       -1   
10        10        0                  0                       -1   
11        11        0                  0                       -1   
12        12        0                  0                        1   
13        13        0             

In [7]:
@njit(parallel=True)
def compute_triplet_imbalance(df_values, comb_indices):
    num_rows = df_values.shape[0]
    num_combinations = len(comb_indices)
    imbalance_features = np.empty((num_rows, num_combinations))

    for i in prange(num_combinations):
        a, b, c = comb_indices[i]
        for j in range(num_rows):
            max_val = max(df_values[j, a], df_values[j, b], df_values[j, c])
            min_val = min(df_values[j, a], df_values[j, b], df_values[j, c])
            mid_val = df_values[j, a] + df_values[j, b] + df_values[j, c] - min_val - max_val
            if mid_val == min_val:
                imbalance_features[j, i] = np.nan
            else:
                imbalance_features[j, i] = (max_val - mid_val) / (mid_val - min_val)

    return imbalance_features

def calculate_triplet_imbalance_numba(price, df):
    df_values = df[price].values
    comb_indices = [(price.index(a), price.index(b), price.index(c)) for a, b, c in combinations(price, 3)]
    features_array = compute_triplet_imbalance(df_values, comb_indices)
    columns = [f"{a}_{b}_{c}_imb2" for a, b, c in combinations(price, 3)]
    features = pd.DataFrame(features_array, columns=columns)

    return features

def imbalance_features(df):
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]

    df["volume"] = df.eval("ask_size + bid_size")
    df["mid_price"] = df.eval("(ask_price + bid_price) / 2")
    df["liquidity_imbalance"] = df.eval("(bid_size-ask_size)/(bid_size+ask_size)")
    df["matched_imbalance"] = df.eval("(imbalance_size-matched_size)/(matched_size+imbalance_size)")
    df["size_imbalance"] = df.eval("bid_size / ask_size")

    for c in combinations(prices, 2):
        df[f"{c[0]}_{c[1]}_imb"] = df.eval(f"({c[0]} - {c[1]})/({c[0]} + {c[1]})")

    for c in [['ask_price', 'bid_price', 'wap', 'reference_price'], sizes]:
        triplet_feature = calculate_triplet_imbalance_numba(c, df)
        df[triplet_feature.columns] = triplet_feature.values

    df["imbalance_momentum"] = df.groupby(['stock_id'])['imbalance_size'].diff(periods=1) / df['matched_size']
    df["price_spread"] = df["ask_price"] - df["bid_price"]
    df["spread_intensity"] = df.groupby(['stock_id'])['price_spread'].diff()
    df['price_pressure'] = df['imbalance_size'] * (df['ask_price'] - df['bid_price'])
    df['market_urgency'] = df['price_spread'] * df['liquidity_imbalance']
    df['depth_pressure'] = (df['ask_size'] - df['bid_size']) * (df['far_price'] - df['near_price'])
    df['reference_price_change'] = df.groupby('stock_id')['reference_price'].pct_change()
    df['imbalance_effect'] = df.groupby('stock_id')['imbalance_size'].pct_change()
    df['price_elasticity'] = df['reference_price_change'] / df['imbalance_effect']
    for func in ["mean", "std", "skew", "kurt"]:
        df[f"all_prices_{func}"] = df[prices].agg(func, axis=1)
        df[f"all_sizes_{func}"] = df[sizes].agg(func, axis=1)
    return df.replace([np.inf, -np.inf], 0).fillna(0)


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
def auto_scaler(data, scaler, column_name):
    untouchables = set(['date_id', 'seconds_in_bucket', 'time_id'])
    # Calculate quartiles
    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    IQR = Q3 - Q1

    # Calculate where data is concentrated within IQR
    lower_concentration = np.mean(data < Q1 + 0.25 * IQR)
    upper_concentration = np.mean(data > Q3 - 0.25 * IQR)

    # Check if the distribution is categorical
    unique_values_count = data.nunique()

    # Categorize distributions
    ########################

    # Categorical
    if unique_values_count <= 10 or column_name in untouchables:
        return data
    # Symmetrical
    elif lower_concentration > 0.7 and upper_concentration > 0.7:
        return scaler.fit_transform(data)
    # Left-Skewed
    elif lower_concentration > 0.7:
        return np.log1p(data)
    # Right-Skewed
    elif upper_concentration > 0.7:
        return np.log1p(data)
    else:
        return data

# Big Testing sets:
split_index = int(len(data) * 0.9)
# Split the data into training and testing sets
global_train_data = data.iloc[:split_index]  # First 90% as training data
global_test_data = data.iloc[split_index:]   # Last 10% as testing data

# Create sequences for each stock
stock_dfs_train = {}
stock_dfs_test = {}

def stock_partition(stock_dict, df):
  for stock_id, group in df.groupby('stock_id'):
      stock_data = group.copy()
      y = stock_data['target']
      X = stock_data.drop('target', axis=1)
      X = imbalance_features(X)
      X['rolling_mean'] = X['wap'].rolling(window=10, min_periods=1).mean()
      X['rolling_std'] = X['wap'].rolling(window=10, min_periods=1).std()
      X.drop(columns=['stock_id'], inplace=True)
      #for column in X.columns:
        #X[column] = auto_scaler(X[column], scaler, column)
      X_norm = scaler.fit_transform(X)
      X_norm = pd.DataFrame(X_norm, columns=X.columns)
      X_norm = reduce_memory_usage(X_norm)
      stock_dict[stock_id] = (X_norm, y)

stock_partition(stock_dfs_train, global_train_data)
stock_partition(stock_dfs_test, global_test_data)

In [9]:
for _, df_set in stock_dfs_train.items():
  df_set[0]['rolling_std'][0] = 0
stock_dfs_train[0][0].info()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23837 entries, 0 to 23836
Data columns (total 63 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   date_id                                    23837 non-null  float16
 1   seconds_in_bucket                          23837 non-null  float16
 2   imbalance_size                             23837 non-null  float16
 3   reference_price                            23837 non-null  float16
 4   matched_size                               23837 non-null  float16
 5   far_price                                  23837 non-null  float16
 6   near_price                                 23837 non-null  float16
 7   bid_price                                  23837 non-null  float16
 8   bid_size                                   23837 non-null  float16
 9   ask_price                                  23837 non-null  float16
 10  ask_size              

In [ ]:
# create time series
# one hot encode categorical features?
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras import
from tensorflow.keras.models import load_model
# Dictionary to store time series data for each stock
stock_time_series_train = {}
stock_time_series_test = {}

# Create sequences for each stock DataFrame
for stock_id, (X_norm, y) in stock_dfs_train.items():
    sequence_length = 30 # Set your desired sequence length here

    # Convert DataFrame to numpy array
    data_array = X_norm.values

    # Define the TimeseriesGenerator for the current stock's data
    generator = TimeseriesGenerator(data_array, y.values,
                                    length=sequence_length, batch_size=2)

    # Store the generated sequences for the stock in the dictionary
    stock_time_series_train[stock_id] = generator


for stock_id, (X_norm, y) in stock_dfs_test.items():
    sequence_length = 30  # Set your desired sequence length here

    # Convert DataFrame to numpy array
    data_array = X_norm.values

    # Define the TimeseriesGenerator for the current stock's data
    generator = TimeseriesGenerator(data_array, y.values,
                                    length=sequence_length, batch_size=2)

    # Store the generated sequences for the stock in the dictionary
    stock_time_series_test[stock_id] = generator


# Dictionary containing LSTM models for each stock
lstm_models = {}

# Define LSTM model architecture with Elastic Net regularization
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(64, input_shape=input_shape, return_sequences=True, kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.01)))
    model.add(LSTM(32, return_sequences=True, kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.01)))
    model.add(LSTM(16, kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.01)))
    model.add(Dense(1, kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.01)))  # Assuming a single output for regression
    model.compile(optimizer='adam', loss='mean_absolute_error')  # Compile the model
    return model

mae_matrix = [[0] * 200] * 200
# Train LSTM models for each stock using corresponding TimeseriesGenerator
for stock_id, generator in stock_time_series_train.items():
    X, y = generator[0]  # Get the first batch from the TimeseriesGenerator

    # Create and compile an LSTM model based on the input shape
    model = create_lstm_model(input_shape=(X.shape[1], X.shape[2]))

    # Train the LSTM model using the TimeseriesGenerator
    model.fit(generator, epochs=1)  # Train for a specified number of epochs (adjust as needed)
    # Store the trained model in the dictionary of LSTM models
    lstm_models[stock_id] = model
    save_path = '/content/drive/MyDrive/ai_ml_suite/models/' + f'stock_{stock_id}_model'
    model.save(save_path)



    # Calculate performance of stock_id model on all other stock test datasets as you go
'''
    for stock_id_t, generator_t in stock_time_series_test.items():
      predictions =  lstm_models[stock_id].predict(generator_t)
      test_values = stock_dfs_test[stock_id_t][1]
      mae_matrix[stock_id][stock_id_t] = mean_absolute_error(predictions, test_values)
    print(mae_matrix[stock_id])

print(mae_matrix)
'''

9483/9483 [==============================] - 75s 7ms/step - loss: 6.9888


'\n    for stock_id_t, generator_t in stock_time_series_test.items():\n      predictions =  lstm_models[stock_id].predict(generator_t)\n      test_values = stock_dfs_test[stock_id_t][1]\n      mae_matrix[stock_id][stock_id_t] = mean_absolute_error(predictions, test_values)\n    print(mae_matrix[stock_id])\n\nprint(mae_matrix)\n'

In [10]:
# create time series
# one hot encode categorical features?
from keras.preprocessing.sequence import TimeseriesGenerator
from keras import regularizers
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
import os
import sys

# Dictionary to store time series data for each stock
stock_time_series_train = {}
stock_time_series_test = {}

# Create sequences for each stock DataFrame
for stock_id, (X_norm, y) in stock_dfs_train.items():
    sequence_length = 30 # Set your desired sequence length here

    # Convert DataFrame to numpy array
    data_array = X_norm.values

    # Define the TimeseriesGenerator for the current stock's data
    generator = TimeseriesGenerator(data_array, y.values,
                                    length=sequence_length, batch_size=2)

    # Store the generated sequences for the stock in the dictionary
    stock_time_series_train[stock_id] = generator


for stock_id, (X_norm, y) in stock_dfs_test.items():
    sequence_length = 30  # Set your desired sequence length here

    # Convert DataFrame to numpy array
    data_array = X_norm.values

    # Define the TimeseriesGenerator for the current stock's data
    generator = TimeseriesGenerator(data_array, y.values,
                                    length=sequence_length, batch_size=2)

    # Store the generated sequences for the stock in the dictionary
    stock_time_series_test[stock_id] = generator



<ipython-input-32-ff93d541034e>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(np.where(np.isnan(stock_time_series_test[0][0])))


ValueError: ignored

In [24]:
mae_matrix = [[0] * 200] * 200
for stock_id, _ in stock_time_series_train.items():
    # Load the saved model
    load_path = '/content/drive/MyDrive/ai_ml_suite/models/' + f'stock_{stock_id}_model'
    loaded_model = load_model(load_path)
    for stock_id_t, generator_t in stock_time_series_test.items():
        predictions =  loaded_model.predict(generator_t, verbose = 0)
        test_values = stock_dfs_test[stock_id_t][1]
        predictions[0, 0] = 0
        predictions.reshape(-1)
        mae_matrix[stock_id][stock_id_t] = mean_absolute_error(predictions, test_values[30:])
    print(mae_matrix[stock_id])

[3.838506471460681, 8.237922376589344, 7.344406512771096, 3.422560340812339, 3.8779162053745155, 7.484347450380498, 5.807141571605657, 5.20861188781559, 5.386618822015149, 4.817477981012948, 4.337504232105186, 8.917055721807435, 4.358399898504899, 4.863627886003995, 4.872970891525781, 5.4074375847229845, 6.376097522105153, 5.539346399876801, 7.899883872819702, 7.32407470449421, 5.7551340839764755, 4.926092981216791, 6.259388896937658, 4.8273319624791045, 4.054582518845952, 5.099932816043443, 5.326314383858208, 5.498535960259689, 5.126750627613919, 4.70249465425385, 4.296448699961247, 22.2094383174026, 3.8867272634951147, 8.224704241438808, 5.490901647506976, 4.231745347602765, 5.444335002485541, 3.7055359734963034, 4.041102345780642, 4.114837960301672, 6.41878649456576, 5.335235255024525, 8.968176321695248, 5.649731841243471, 4.498327922995066, 3.198736151600114, 5.423068127467342, 5.189482422160702, 5.015470084605688, 6.018949923782383, 6.796638001123765, 7.1682417475999864, 4.6060844

KeyboardInterrupt: 

In [ ]:
# prompt: assuming I've already mounted my drive, create file and save it in my google drive

with open('/content/drive/MyDrive/filename.txt', 'w') as f:
  f.write('Hello world!')
